In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

med_dataset = "Thaweewat/thai-med-pack"

In [4]:
huggingface_dataset = load_dataset(med_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/189190 [00:00<?, ? examples/s]

In [5]:
huggingface_dataset

Dataset({
    features: ['text'],
    num_rows: 189190
})

In [6]:
import pandas as pd
dataset_pd = pd.DataFrame(huggingface_dataset)

In [7]:
dataset_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189190 entries, 0 to 189189
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    189190 non-null  object
dtypes: object(1)
memory usage: 1.4+ MB


In [8]:
from datasets import Dataset

# สมมุติว่า df เป็น DataFrame ที่คุณได้แก้ไขแล้ว
customized_dataset = Dataset.from_pandas(dataset_pd)

In [9]:
customized_dataset[18000]

{'text': '<s>[INST] อยากทราบว่ามีคนมาลักหลับจะรู้สึกตัวไหมคะ ในกรณีผู้หญิง ไม่ได้เมา ไม่ได้ถูกวางยาหรือกินยาที่มีฤทธิ์ง่วงแต่อย่างใด [/INST] สวัสดีค่ะ หากไม่ได้ถูกให้ทานยาที่มีฤทธิ์กดประสาท ทำให้ง่วงซึมหรือหลงลืมใดๆ การโดนสอดใส่อวัยวะเพศเข้าสู่ช่องคลอดโดยไม่ได้มีอารมณ์ร่วมนั้น ย่อมต้องมีอาการเจ็บ เพราะช่องคลอดไม่ได้มีน้ำหล่อลื่นออกมา ซึ่งน่าจะทำให้เรารู้สึกตัวสะดุ้งตื่นขึ้น ดังนั้น จึงไม่น่าเป็นไปได้ที่จะโดนสอดใส่โดยไม่รู้ตัวค่ะ </s>'}

In [10]:
def transform_structure(example):
    # Extract and split the text
    instruction_text = example['text'].split('[INST] ')[1].split(' [/INST]')[0]
    response_text = example['text'].split('[INST] ')[1].split(' [/INST]')[1].strip()

    # Strip off the ending token </s> if it's present
    if response_text.endswith('</s>'):
        response_text = response_text[:-4].strip()

    # Return the new structure
    return {
        'instruction': instruction_text,
        'input': '',
        'output': response_text,
        'text': ''
    }

# Apply the transformation to each record in the dataset again
transformed_dataset = customized_dataset.map(transform_structure)


Map:   0%|          | 0/189190 [00:00<?, ? examples/s]

In [11]:
customized_dataset[18000]

{'text': '<s>[INST] สวัสดีค่ะ หนูมีอาการท้องป่องแต่ไม่มีอาการปวดใดๆ ประมาณเดือนนึงแล้วหนูสังเกตเวลานอนหงายท้องน้อยขวาล่างมีอาการบีบตัวรู้สึกได้ชัดเจน คลำดูมีลักษณะเหมือนลำไส้บีบตัวแข็งขึ้นความยาวขนาดนิ้วกลางค่ะ บีบแข็งคลำได้เป็นก้อนแล้วก็คลายออก หนูไม่เจ็บเลยค่ะแต่รู้สึกตึงๆ อาการบีบแข็งนี้ถ้านอนหงายจะรู้สึกเป็นระยะห่างประมาณ10-15นาทีต่อครั้งจนหนูต้องนอนตะแคงขวาถึงนอนหลับได้ค่ะ ถ้านอนหงายรู้สึกเหมือนมีก้อนหินหนักซักร้อยกรัมอยู่ตรงบริเวณนั้นเลยค่ะ เวลากลางวันจะรู้สึกบีบเกร็งแม้กระทั่งยืนหรือเดินเหมือนมีก้อนดันขึ้นบริเวณหัวกางเกงแบบนี้ค่ะ แล้วก็ไม่เจ็บแต่จะเป็นอาการแบบทำให้รู้สึกกังวล หนูอยู่อังกฤษตอนนี้กำลังตรวจหาสาเหตุอยู่ค่ะ อาทิตย์ก่อนตรวจเนื้อบอกมดลูกแต่ผลก็ปกติ แล้วตรวจเลือดก็ปกติ วันนี้ให้งดอาหาร12ชั่วโมงแล้วตรวจเลือดผลจะออกอาทิตย์หน้า หนูอยากทราบว่าอาการที่หนูเป็นคืออะไรคะ [/INST] สวัสดีค่ะ อาการท้องโตขึ้น อาจเกิดจากการมีน้ำหรือลมในช่องท้องมากเกินไป เช่นการรับประทานอาหารที่มีแก๊ซมากหรืออาหารย่อยยากมีไขมันสูง หรือเกิดจากมีก้อนผิดปกติในช่องท้อง จากมดลูก รังไข่ กระเพาะปัสสาวะ หรือลำ

In [12]:
transformed_dataset[18000]

{'text': '',
 'instruction': 'สวัสดีค่ะ หนูมีอาการท้องป่องแต่ไม่มีอาการปวดใดๆ ประมาณเดือนนึงแล้วหนูสังเกตเวลานอนหงายท้องน้อยขวาล่างมีอาการบีบตัวรู้สึกได้ชัดเจน คลำดูมีลักษณะเหมือนลำไส้บีบตัวแข็งขึ้นความยาวขนาดนิ้วกลางค่ะ บีบแข็งคลำได้เป็นก้อนแล้วก็คลายออก หนูไม่เจ็บเลยค่ะแต่รู้สึกตึงๆ อาการบีบแข็งนี้ถ้านอนหงายจะรู้สึกเป็นระยะห่างประมาณ10-15นาทีต่อครั้งจนหนูต้องนอนตะแคงขวาถึงนอนหลับได้ค่ะ ถ้านอนหงายรู้สึกเหมือนมีก้อนหินหนักซักร้อยกรัมอยู่ตรงบริเวณนั้นเลยค่ะ เวลากลางวันจะรู้สึกบีบเกร็งแม้กระทั่งยืนหรือเดินเหมือนมีก้อนดันขึ้นบริเวณหัวกางเกงแบบนี้ค่ะ แล้วก็ไม่เจ็บแต่จะเป็นอาการแบบทำให้รู้สึกกังวล หนูอยู่อังกฤษตอนนี้กำลังตรวจหาสาเหตุอยู่ค่ะ อาทิตย์ก่อนตรวจเนื้อบอกมดลูกแต่ผลก็ปกติ แล้วตรวจเลือดก็ปกติ วันนี้ให้งดอาหาร12ชั่วโมงแล้วตรวจเลือดผลจะออกอาทิตย์หน้า หนูอยากทราบว่าอาการที่หนูเป็นคืออะไรคะ',
 'input': '',
 'output': 'สวัสดีค่ะ อาการท้องโตขึ้น อาจเกิดจากการมีน้ำหรือลมในช่องท้องมากเกินไป เช่นการรับประทานอาหารที่มีแก๊ซมากหรืออาหารย่อยยากมีไขมันสูง หรือเกิดจากมีก้อนผิดปกติในช่องท้อง จากมดลู

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "openthaigpt/openthaigpt-1.0.0-beta-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.27G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/718k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        # Add other relevant submodules from LlamaDecoderLayer here
    ]
)

In [17]:
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/llm_ThaiMed_checkpoints/thai-med-pack"
per_device_train_batch_size = 2 ## จากเดิม 4
gradient_accumulation_steps = 2 ## จากเดิม 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [18]:
def formatting_prompts_func(examples):
    output_text = []
    for i in range(len(examples)):
        instruction = examples["instruction"][i]
        response = examples["output"][i]
        text = f'<s>[INST] <<SYS>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>{instruction}[/INST]{response}'
        output_text.append(text)
    return output_text

In [19]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(41088, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [20]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConf

Map:   0%|          | 0/189190 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [21]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,3.102400
20,2.845400
30,2.629500
40,2.416600
50,2.360100
60,2.559700
70,2.439700
80,2.396300
90,2.345900
100,2.402900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=100, training_loss=2.549840145111084, metrics={'train_runtime': 197.2924, 'train_samples_per_second': 2.027, 'train_steps_per_second': 0.507, 'total_flos': 9854454100193280.0, 'train_loss': 2.549840145111084, 'epoch': 0.5263157894736842})

### Model Inference

In [23]:
!nvidia-smi

Sat Jun  8 12:10:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0              54W / 400W |  25099MiB / 40960MiB |      3%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [24]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "openthaigpt/openthaigpt-1.0.0-beta-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
custom_prompt = "โปรดอธิบายลักษณะช่องปากที่เป็นมะเร็งในระยะเริ่มต้น"

In [27]:
PROMPT =f'<s>[INST] <<SYS>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>{custom_prompt}[/INST]'

In [28]:
%%time
from peft import PeftModel
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers import GenerationConfig

model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/llm_ThaiMed_checkpoints/thai-med-pack/checkpoint-100")

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

Generating...
<s><s>[INST] <<SYS>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>โปรดอธิบายลักษณะช่องปากที่เป็นมะเร็งในระยะเริ่มต้น[/INST]ช่องปากเป็นเนื้องอกที่โตขึ้นและอาจทำให้เกิดอาการเจ็บปวดหรือมีเลือดออกเมื่อกัดฟัน มีก้อนที่บวมแดงหรือมีแผลเกิดขึ้นในช่องปาก และอาจมีการสูญเสียน้ำหนัก การเปลี่ยนแปลงของเสียงพูด หายใจลำบาก และปัญหาในการเคี้ยวอาหาร นอกจากนี้ อาจพบการเปลี่ยนแปลงของต่อมน้ำลาย ฝีที่คอ ปวดศีรษะ คลื่นไส้ อาเจียน น้ำหนักลดโดยไม่ทราบสาเหตุ และอ่อนเพลีย อาการอื่นๆ ที่พบได้น้อย ได้แก่ ความดันโลหิตสูง ท้องผูก และภาวะติดเชื้อทางเดินปัสสาวะ หากคุณสงสัยว่าตนเองอาจเป็นมะเร็งช่องปาก โปรดปรึกษาแพทย์เพื่อรับการวินิจฉัยและการรักษาที่เหมาะสมต่อไป</s><s> #1423765 - 08/19/19 07:38 PM Re: The New Horse Barn [Re: johnny c]
I would think it's not really that far off from what you have now, but IMO it does look like it could be improved upon. It seems to lack any real character 

In [29]:
import torch
torch.cuda.empty_cache()

In [30]:
custom_prompt = "ถูกงูพิษกัดต้องทำอย่างไรครับ"
PROMPT =f'<s>[INST] <<SYS>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>{custom_prompt}[/INST]'

In [31]:
inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
<s><s>[INST] <<SYS>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>ถูกงูพิษกัดต้องทำอย่างไรครับ[/INST]สวัสดีคะ งูมีพิษต่อระบบประสาทค่ะ หากโดนพิษแล้วจะมีอาการปวดแสบ ปวดร้อนที่แผล มีรอยแดงๆรอบบริเวณที่ถูกกัด และอาจมีอาการอื่นๆร่วมด้วย เช่น คลื่นไส้ อาเจียน มึนงง พูดไม่ชัด ใจสั่น น้ำตาไหล เป็นต้นค่ะ การดูแลเบื้องต้น ควรล้างแผลด้วยน้ำสะอาดและสบู่หลายๆครั้งเพื่อลดปริมาณพิษให้มากที่สุด จากนั้นไปพบแพทย์เพื่อรับการฉีดยาต้านพิษค่ะ นอกจากนี้ควรสังเกตอาการต่างๆดังที่กล่าวมาข้างต้น หากมีอาการผิดปกติใดๆ ให้รีบกลับไปพบแพทย์ทันทีค่ะ เพราะหากเกิดภาวะแทรกซ้อนจะทำให้การรักษายากขึ้นค่ะ หวังว่านี่จะเป็นข้อมูลที่เป็นประโยชน์นะคะ ขอบคุณค่ะ ฉันหวังว่าฉันได้ตอบคำถามของคุณแล้ว โปรดอย่าลังเลที่จะติดต่อเราหากคุณต้องการความช่วยเหลือเพิ่มเติมหรือมีข้อสงสัยเกี่ยวกับคำแนะนำนี้ ขอแสดงความนับถือ,พญ.วริสรา เชื้อวิริยะกุล (พญ.วริสรา เชื้อวิริยะกุล) แพทย์เฉพาะทางเวชศาสตร์ฉุกเฉิน โรงพยาบาลพญ

In [32]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(41088, 5120)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l